Ce script a servi à extraire les données de l'arborescence des fichiers, ensuite séparer les fichiers en base de données amplicon et variant caller.

In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import tqdm
from scipy.stats import zscore

In [2]:
# all file paths
from glob import glob
paths = glob(r"C:\Users\ahmed\MFE project\Final\**", recursive= True)

In [3]:
pathy = [i for i in paths if (len(i)>=120) & (i[-3:]!="pdf")]
pathy_amplicon = [i for i in pathy if i[-16:-8]=="amplicon"]
amplicon_liste = [i.split("\\")[-1] for i in pathy_amplicon]

pathys = [i for i in paths if (i[-3:]!="pdf") & (len(i)>60) & (i.find("bcmatrix")<0)]
pathy_tsv = [i for i in pathys if i[-3:]=="tsv"]
liste = [i.split("\\")[-1] for i in pathy_tsv]

ampli_liste = [i.split('_')[1] for i in amplicon_liste]
tsv_liste = [i.split('_')[1] for i in liste]


# je print pour voir s'ils ont la même taille
print(len(ampli_liste), len(tsv_liste))

608 608


In [4]:
# vérifier qu'ils ont bien le même ordre
for i in range(len(ampli_liste)):
    if ampli_liste[i] != tsv_liste[i]:
        print(i,ampli_liste[i])

In [5]:
pathy_amplicon


['C:\\Users\\ahmed\\MFE project\\Final\\2018\\NDN_4-12-18_1\\OPT\\IonXpress_007_Auto_NDN_04-12-18_1_s5-torrent-server-vm_83.amplicon.cov.xls',
 'C:\\Users\\ahmed\\MFE project\\Final\\2018\\NDN_4-12-18_1\\OPT\\IonXpress_008_Auto_NDN_04-12-18_1_s5-torrent-server-vm_83.amplicon.cov.xls',
 'C:\\Users\\ahmed\\MFE project\\Final\\2018\\NDN_4-12-18_1\\SOPT\\IonXpress_005_Auto_NDN_04-12-18_1_s5-torrent-server-vm_83.amplicon.cov.xls',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\NC\\IonXpress_083_Auto_CVC_02_04_19_1_s5-torrent-server-vm_137.amplicon.cov.xls',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\NVA\\IonXpress_075_Auto_CVC_02_04_19_1_s5-torrent-server-vm_137.amplicon.cov.xls',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\NVA\\IonXpress_080_Auto_CVC_02_04_19_1_s5-torrent-server-vm_137.amplicon.cov.xls',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\OPT\\IonXpress_084_Auto_CVC_02_04_19_1_s5-torrent-server-vm_137.amplicon.c

In [6]:
pathy_tsv

['C:\\Users\\ahmed\\MFE project\\Final\\2018\\NDN_4-12-18_1\\OPT\\IonXpress_007_18M03649.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2018\\NDN_4-12-18_1\\OPT\\IonXpress_008_18M03651.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2018\\NDN_4-12-18_1\\SOPT\\IonXpress_005_18M03648.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\NC\\IonXpress_083_L19MET3.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\NVA\\IonXpress_075_L19MOL1.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\NVA\\IonXpress_080_L19MOL6.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\OPT\\IonXpress_084_19M00820.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\OPT\\IonXpress_085_19M00821.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\OPT\\IonXpress_086_19M00826.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\CVC_02-04-19_1\\SOPT\\IonXpress_081_L19MET1.tsv',
 'C:\\Users\\ahmed\\MFE project\\Final\\2019\\

In [7]:
# extract information from tsv file
# caract = ["annee", "type", "date", "num", "classe"]
def extract_info_tsv(path):
    split = path.split('\\')
    annee = split[5]
    sp = split[6]
    spi = sp.replace('_', '-')
    spl = spi.split('-')
    type = 0
    date = 0
    num = 0
    if len(spl)==5:
        type = spl[0]
        date = '-'.join(spl[1:-1])
        num = spl[-1]
    if len(spl)==6:
        type = spl[0]
        date = spl[2]
        num = spl[-1]
    classe = split[7]
    return [annee, type, date, num, classe]

In [95]:
# create the csv file tsv version
caract = ["annee", "type", "date", "num", "classe"]
df_list=[]
for j in range(len(pathy_tsv)):
    path = pathy_tsv[j]
    dfa = pd.read_csv(path, sep='\t')
    if str(dfa["Barcode"][0])!="nan":
        df1 = dfa[dfa["Barcode"]==dfa["Barcode"][0]]
        df1.index = range(len(df1))
        cara = extract_info_tsv(path)
        for i in range(len(caract)):
            df1.loc[:,caract[i]]= cara[i]
        df1.loc[:,'ordre'] = j
        df1.loc[:,"longueur"] = len(df1)
        df_list.append(df1)
df_tsv = pd.concat(df_list)
df_tsv.index = range(len(df_tsv))
# df_tsv.to_csv('Database_tsv.csv')

In [92]:
# extract information about the sample to complete the ngs result
caract = ["annee", "type", "date", "num", "classe", "num2", "num3"]
def extract_info_amplicon(path):
    split = path.split('\\')
    annee = split[5]
    sp = split[6]
    spi = sp.replace('_', '-')
    spl = spi.split('-')
    type = 0
    date = 0
    num = 0
    if len(spl)==5:
        type = spl[0]
        date = '-'.join(spl[1:-1])
        num = spl[-1]
    if len(spl)==6:
        type = spl[0]
        date = spl[2]
        num = spl[-1]
    classe = split[7]
    num2 = split[-1].split('_')[1]
    num3 = split[-1].split('_')[-1][:-17]
    return [annee, type, date, num, classe, num2, num3]

# def extract_info_chr(path):
#     split = path.split('\\')
#     annee = split[5]
#     sp = split[6]
#     spi = sp.replace('_', '-')
#     spl = spi.split('-')
#     type = 0
#     date = 0
#     num = 0
#     if len(spl)==5:
#         type = spl[0]
#         date = '-'.join(spl[1:-1])
#         num = spl[-1]
#     if len(spl)==6:
#         type = spl[0]
#         date = spl[2]
#         num = spl[-1]
#     classe = split[7]
#     num2 = split[-1].split('_')[1]
#     num3 = split[-1].split('_')[-1][:-12]
#     return [annee, type, date, num, classe, num2, num3]

In [94]:
# create the csv file amplicon version
caract = ["annee", "type", "date", "num", "classe", "num2", "num3"]
df_list=[]
for j in range(len(pathy_amplicon)):
    path = pathy_amplicon[j]
    df1 = pd.read_csv(path, sep='\t')
    cara = extract_info_amplicon(path)
    for i in range(len(caract)):
        df1.loc[:,caract[i]]= cara[i]
    df1.loc[:,'ordre'] = j
    df1.loc[:, "longueur"] = len(df1)
    df_list.append(df1)
df_amplicon = pd.concat(df_list)
df_amplicon.index = range(len(df_amplicon))
df_amplicon.to_csv('Database_Amplicon.csv')

In [73]:
# # create the csv file chromosome version
# caract = ["annee", "type", "date", "num", "classe", "num2", "num3"]
# df_list=[]
# for j in range(len(pathy_chr)):
#     path = pathy_chr[j]
#     df1 = pd.read_csv(path, sep='\t')
#     cara = extract_info_chr(path)
#     for i in range(len(caract)):
#         df1.loc[:,caract[i]]= cara[i]
#     df1.loc[:,'ordre'] = j
#     df_list.append(df1)
# df_chr = pd.concat(df_list)
# df_chr.index = range(len(df_chr))
# df_chr.to_csv('Database_Chr.csv')